In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import plotly.express as px
import statistics
from datetime import datetime, date, timedelta

In [ ]:
#Read the data into a data frame
df = pd.read_csv("extract-3-very-clean.csv")

In [ ]:
#See how many records are included
df.size

In [ ]:
#Change date fields to datetime type
df['Contract date']= pd.to_datetime(df['Contract date'])
df['Settlement date']= pd.to_datetime(df['Settlement date'])

#Then check types are okay
df.dtypes

In [ ]:
#Filter the dataset to your own search area
#(could obvs filter by whatever, but this is my search area)

property_locations = ['Lawson','Hazelbrook','Woodford','Linden','Faulconbridge','Springwood','Valley Heights','Warrimoo']
property_streetname = None #e.g.: ['Railway Ave']
exclude_zoning = ['IN1', 'IN2', 'I', 'B', 'B1', 'B2', 'B7']
exclude_primary_purpose = ['Service stations', 'Service stati', 'Service statio', 'Shop', 'Hall', 'Commercial']
include_only_primary_purpose = None #e.g.: ['Vacant land']
postcode_min = 2750
postcode_max = 2800
area_min = 500
area_max = None
start_date = '2016-01-01'
end_date = '2100-01-01'

#Go ahead and implement all of the above filters
df_myarea = df
if property_locations: df_myarea = df_myarea[ df_myarea['Property locality'].isin(property_locations) ] #In location specified
if property_streetname: df_myarea = df_myarea[ df_myarea['Property street name'].isin(property_streetname) ] #In street name exactly specified
if area_min: df_myarea = df_myarea[ df_myarea['Area'] >= area_min ] #More than minimum area size
if area_max: df_myarea = df_myarea[ df_myarea['Area'] <= area_max ] #Less than maximum area size
if postcode_min: df_myarea = df_myarea[ df_myarea['Property post code'] >= postcode_min ] #In postcode range
if postcode_max: df_myarea = df_myarea[ df_myarea['Property post code'] <= postcode_max ] #In postcode range
if exclude_zoning: df_myarea = df_myarea[ ~df_myarea['Zoning'].isin(exclude_zoning) ] #Exclude weird zoning types
if start_date: df_myarea = df_myarea[ df_myarea['Contract date'] >= start_date] #Make sure all values are in the correct date range
if end_date: df_myarea = df_myarea[ df_myarea['Contract date'] <= end_date] #Make sure all values are in the correct date range
if exclude_primary_purpose: df_myarea = df_myarea[ ~df_myarea['Primary purpose'].isin(exclude_primary_purpose) ] #Exclude weird zoning types
if include_only_primary_purpose: df_myarea = df_myarea[ df_myarea['Primary purpose'].isin(include_only_primary_purpose) ] #Include only these zoning types

print(str(len(df_myarea.index)) + ' records kept')

In [ ]:
#Show zoning and purpose types in the dataset
#Types: https://www.valuergeneral.nsw.gov.au/__data/assets/pdf_file/0019/216406/Property_Sales_Data_File_Zone_Codes_and_Descriptions_V2.pdf

display(df_myarea['Primary purpose'].unique())
display(df_myarea['Zoning'].unique())

In [ ]:
#Fix NaNs
df_myarea['Zoning'].fillna(value='None', inplace=True)
df_myarea['Area'].fillna(value=0, inplace=True)

In [ ]:
#Remove purchase price outliers

before=len(df_myarea.index)

#Display the outliers
display(df_myarea[(np.abs(stats.zscore(df_myarea['Purchase price'])) >= 5)])

#Remove them from the data
#df_myarea = df_myarea[(np.abs(stats.zscore(df_myarea['Purchase price'])) < 3)]

after=len(df_myarea.index)
print('Removed ' + str(before-after) + ' outliers (more than 5 standard deviations from the mean).')

In [ ]:
#Price histogram in ~$50K bins
fig = px.histogram(df_myarea, x="Purchase price", nbins=int(df_myarea['Purchase price'].max()/50000),
    title='Price histogram', width=1000, height=400,
)
fig.show()


In [ ]:
#Display all the records.
display(df_myarea)

In [ ]:
#Price by size and contract date

#Scale property size so the dots don't get too small
median = statistics.median(df_myarea['Area'])
df_myarea['Area - scaled'] = [(x - median) / 15 + median for x in df_myarea['Area']]

fig = px.scatter(
    df_myarea,
    x='Contract date',
    y='Purchase price',    
    size='Area - scaled',
    color='Zoning',
    title='Price and size of property by contract date',
    width=1000,
    height=500,
    labels={'x':'Contract date'},
    hover_name=df_myarea['Property house number'] + ' ' + df_myarea['Property street name'] + ', ' + df_myarea['Property locality'],
    hover_data={
        'Area - scaled':False,
        'Zoning':True,
        'Area':True
    }
)

fig.show()

In [ ]:
#Price by contract date

fig = px.scatter(
    df_myarea,
    x='Contract date',
    y='Purchase price',    
    title='Price over time',
    trendline='rolling',
    #trendline_options=dict(window=45),    
    trendline_color_override="red",
    width=1000,
    height=500,
    labels={'x':'Contract date'},
    hover_name=df_myarea['Property house number'] + ' ' + df_myarea['Property street name'] + ', ' + df_myarea['Property locality'],
    hover_data={
        'Area - scaled':False,
        'Zoning':True,
        'Area':True
    }
)

fig.show()

In [ ]:
#Median price by contract date

df_myarea_agg=df_myarea[['Contract date','Purchase price']]
df_myarea_agg=df_myarea_agg.groupby(['Contract date']).median()
#This is the same as above:
##df_myarea_agg=df_myarea_agg.groupby(([pd.Grouper(key='Contract date', freq='D')])).median()

fig = px.scatter(
    df_myarea_agg,
    x=df_myarea_agg.index.values,
    y='Purchase price',    
    title='Median price by date',
    width=1000,
    height=500,
    labels={'x':'Contract date'},
)

fig.show()

In [ ]:
#Get last downloaded file date and set variable 90 days before that
d = datetime.now()
offset = -datetime.now().weekday() - 7 - 90
this_date = (datetime.now() + timedelta(offset))
print(this_date)

In [ ]:
#Monthly median price

df_myarea_aggM = df_myarea[['Contract date', 'Purchase price']]
df_myarea_aggM = df_myarea_aggM.groupby([pd.Grouper(key='Contract date', freq='M')]).agg('median')

df_myarea_aggM['Rolling 6-month median'] = df_myarea_aggM.rolling(6).median()

#Could also do this if we wanted to show multiple types - e.g. mean, sum, etc
#g1 = df_myarea_m.groupby(pd.Grouper(key='Contract date', freq="M")).median()
#g2 = df_myarea_m.groupby(pd.Grouper(key='Contract date', freq="M")).mean()
#g = g1.merge(g2, left_on='Contract date', right_on='Contract date', suffixes=(' median', ' mean'))

fig = px.line(
    df_myarea_aggM,
    title='Monthly median purchase price',
    width=1000,
    height=500
)

fig.add_vline(x=this_date, line_width=2, line_dash="dot", line_color="green")
fig.show()

In [ ]:
#Sales volume by month

latest_date = df_myarea['Contract date'].max() - timedelta(days=90)

df_myarea_aggMc = df_myarea[['Contract date', 'Purchase price']]
df_myarea_aggMc = df_myarea_aggMc.groupby([pd.Grouper(key='Contract date', freq='M')]).agg('count')
df_myarea_aggMc.rename(columns={'Purchase price':'Number of sales'}, inplace=True)
df_myarea_aggMc['Rolling 6-month median'] = df_myarea_aggMc.rolling(6).median()


fig = px.line(
    df_myarea_aggMc,
    title='Sales volume by month',
    width=1000,
    height=500
)

fig.add_vline(x=this_date, line_width=2, line_dash="dot", line_color="green")
fig.show()